In [302]:
import urllib.request
from bs4 import BeautifulSoup
import time

url = 'http://www.boxofficemojo.com/yearly/chart/?view2=worldwide&yr=2017'  
html = urllib.request.urlopen(url)    
bsObj = BeautifulSoup(html,'lxml-xml')
bsObj

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="en">
<HEAD>
<TITLE>2017 Yearly Box Office Results - Box Office Mojo</TITLE>
<META CONTENT="2017, year, yearly, box, office, result, list, movie, movies, listing, listings, top movies, all time, film" NAME="keywords">
<META CONTENT="Yearly box office results for 2017." NAME="description">
<link charset="utf-8" href="/css/mojo.css?1" media="screen" rel="stylesheet" title="no title" type="text/css">
<link charset="utf-8" href="/css/mojo.css?1" media="print" rel="stylesheet" title="no title" type="text/css"/>
<body>
<iframe frameborder="0" height="1" id="sis_pixel_sitewide" marginheight="0" marginwidth="0" style="display: none;" width="1"/>
<script>
    setTimeout(function(){
        try{
            //sis3.0 pixel
            var cacheBust = Math.random() * 10000000000000000,
                url_sis3 = 'http://s.amazon-adsystem.com/iu3?'

In [303]:
href_list=bsObj.findAll('a', attrs={'href': re.compile(r"movies")})
href_list

[<a href="/movies/?id=marvel2017b.htm">#1 Movie: 'Black Panther'</a>,
 <a href="/movies/?id=starwars8.htm"><b>Star Wars: The Last Jedi</b></a>,
 <a href="/movies/?id=beautyandthebeast2017.htm"><b>Beauty and the Beast (2017)</b></a>,
 <a href="/movies/?id=furious8.htm"><b>The Fate of the Furious</b></a>,
 <a href="/movies/?id=despicableme3.htm"><b>Despicable Me 3</b></a>,
 <a href="/movies/?id=jumanji2016.htm"><b>Jumanji: Welcome to the Jungle</b></a>,
 <a href="/movies/?id=spiderman2017.htm"><b>Spider-Man: Homecoming</b></a>,
 <a href="/movies/?id=wolfwarrior2.htm"><b>Wolf Warrior 2</b></a>,
 <a href="/movies/?id=marvel17a.htm"><b>Guardians of the Galaxy Vol. 2</b></a>,
 <a href="/movies/?id=marvel2017.htm"><b>Thor: Ragnarok</b></a>,
 <a href="/movies/?id=wonderwoman.htm"><b>Wonder Woman</b></a>,
 <a href="/movies/?id=potc5.htm"><b>Pirates of the Caribbean: Dead Men Tell No Tales</b></a>,
 <a href="/movies/?id=pixar1117.htm"><b>Coco</b></a>,
 <a href="/movies/?id=it.htm"><b>It</b></a>,

In [306]:
Link=[]
for i in range(1,len(href_list)):
    m=href_list[i].attrs
    x=str(list(m.values())[0])
    y="http://www.boxofficemojo.com"+x
    Link.append(y)
Link


['http://www.boxofficemojo.com/movies/?id=starwars8.htm',
 'http://www.boxofficemojo.com/movies/?id=beautyandthebeast2017.htm',
 'http://www.boxofficemojo.com/movies/?id=furious8.htm',
 'http://www.boxofficemojo.com/movies/?id=despicableme3.htm',
 'http://www.boxofficemojo.com/movies/?id=jumanji2016.htm',
 'http://www.boxofficemojo.com/movies/?id=spiderman2017.htm',
 'http://www.boxofficemojo.com/movies/?id=wolfwarrior2.htm',
 'http://www.boxofficemojo.com/movies/?id=marvel17a.htm',
 'http://www.boxofficemojo.com/movies/?id=marvel2017.htm',
 'http://www.boxofficemojo.com/movies/?id=wonderwoman.htm',
 'http://www.boxofficemojo.com/movies/?id=potc5.htm',
 'http://www.boxofficemojo.com/movies/?id=pixar1117.htm',
 'http://www.boxofficemojo.com/movies/?id=it.htm',
 'http://www.boxofficemojo.com/movies/?id=dcfilm1117.htm',
 'http://www.boxofficemojo.com/movies/?id=wolverine2017.htm',
 'http://www.boxofficemojo.com/movies/?id=transformers5.htm',
 'http://www.boxofficemojo.com/movies/?id=legen

In [307]:
td_list=bsObj.find_all('td') 
td_list

[<td valign="center">
 <font face="Verdana" size="2"><a href="/about/adjuster.htm"><b>Adjuster:</b></a></font>
 <select name="ticketyr" size="1" style="font-family: Verdana; font-size: 10pt">
 <option value="0">Actuals</option>
 <option value="1">Est. Tckts</option>
 <script language="javascript">
   for(i=2018; i&gt;=1933; i--) {
   	document.write('<option value="' + i + '">');
 	if(i=='0') document.write(' selected');
 	document.write('&gt;' + i );
 	if(i=='0') document.write(', $' + '0.00');
 	document.write('</option>');
   }
 </script>
 <option value="1929">1929</option>
 <option value="1924">1924</option>
 <option value="1910">1910</option>
 </select><input name="Go" style="font-size: 10pt; height: 22" type="submit" value="Go">
 </input></td>,
 <td bgcolor="#8b0000" colspan="2"><img border="0" height="5" src="/images/space.gif" width="1"/></td>,
 <td align="center" valign="top"><br>
 <h1>2017 WORLDWIDE GROSSES</h1><p/></br></td>,
 <td align="center"><font size="2"><a href="/year

In [308]:
data=[]
for td in td_list:
    l=[]
    for tr in td:
        l.append(tr.string)
    data.append(l)

In [309]:
data

[['\n', 'Adjuster:', '\n', None, '\n'],
 [None],
 [None],
 ['Rank'],
 ['Title (click to view)'],
 [None, 'Worldwide', None, None],
 ['\n', None],
 ['Worldwide'],
 [None, '%'],
 [None, '%'],
 ['1'],
 ['Star Wars: The Last Jedi'],
 ['BV'],
 ['$1,331.1'],
 ['$619.1'],
 ['46.5%'],
 ['$712.0'],
 ['53.5%'],
 ['2'],
 ['Beauty and the Beast (2017)'],
 ['BV'],
 ['$1,263.5'],
 ['$504.0'],
 ['39.9%'],
 ['$759.5'],
 ['60.1%'],
 ['3'],
 ['The Fate of the Furious'],
 ['Uni.'],
 ['$1,236.0'],
 ['$226.0'],
 ['18.3%'],
 ['$1,010.0'],
 ['81.7%'],
 ['4'],
 ['Despicable Me 3'],
 ['Uni.'],
 ['$1,034.8'],
 ['$264.6'],
 ['25.6%'],
 ['$770.2'],
 ['74.4%'],
 ['5'],
 ['Jumanji: Welcome to the Jungle'],
 ['Sony'],
 ['$930.0'],
 ['$393.1'],
 ['42.3%'],
 ['$536.9'],
 ['57.7%'],
 ['6'],
 ['Spider-Man: Homecoming'],
 ['Sony'],
 ['$880.2'],
 ['$334.2'],
 ['38.0%'],
 ['$546.0'],
 ['62.0%'],
 ['7'],
 ['Wolf Warrior 2'],
 ['HC'],
 ['$870.3'],
 ['$2.7'],
 ['0.3%'],
 ['$867.6'],
 ['99.7%'],
 ['8'],
 ['Guardians of the Gal

In [310]:
import numpy as np
I = [0,1,2,3,4,5,6,7,8,9]
data=np.delete(data, I).tolist()

In [311]:
Rank=[]
Title=[]
Studio=[]
Worldwide=[]
Domestic=[]
Overseas=[]
for i in range(len(data)):
    col=i%8
    x=data[i][0]
    if col==0:
        Rank.append(x)
    elif col==1:
        Title.append(x)
    elif col==2:
        Studio.append(x)
    elif col==3:
        Worldwide.append(x)
    elif col==4:
        Domestic.append(x)
    elif col==6:
        Overseas.append(x)
Rank.pop() 
Title.pop() 


In [333]:
import pandas as pd
dict={'Rank':Rank,
      'Title':Title,
      'Studio':Studio,
      'Worldwide':Worldwide,
      'Domestic':Domestic,
      'Overseas':Overseas,
      'Link':Link}

boxoffice = pd.DataFrame(dict)                   
boxoffice = boxoffice[['Rank', 'Title', 'Studio','Worldwide','Domestic','Overseas','Link']]
boxoffice

,Rank,Title,Studio,Worldwide,Domestic,Overseas,Link
0,1,Star Wars: The Last Jedi,BV,"$1,331.1",$619.1,$712.0,http://www.boxofficemojo.com/movies/?id=starwa...
1,2,Beauty and the Beast (2017),BV,"$1,263.5",$504.0,$759.5,http://www.boxofficemojo.com/movies/?id=beauty...
2,3,The Fate of the Furious,Uni.,"$1,236.0",$226.0,"$1,010.0",http://www.boxofficemojo.com/movies/?id=furiou...
3,4,Despicable Me 3,Uni.,"$1,034.8",$264.6,$770.2,http://www.boxofficemojo.com/movies/?id=despic...
4,5,Jumanji: Welcome to the Jungle,Sony,$930.0,$393.1,$536.9,http://www.boxofficemojo.com/movies/?id=jumanj...
5,6,Spider-Man: Homecoming,Sony,$880.2,$334.2,$546.0,http://www.boxofficemojo.com/movies/?id=spider...
6,7,Wolf Warrior 2,HC,$870.3,$2.7,$867.6,http://www.boxofficemojo.com/movies/?id=wolfwa...
7,8,Guardians of the Galaxy Vol. 2,BV,$863.8,$389.8,$473.9,http://www.boxofficemojo.com/movies/?id=marvel...
8,9,Thor: Ragnarok,BV,$853.8,$315.0,$538.8,http://www.boxofficemojo.com/movies/?id=marvel...
9,10,Wonder Woman,WB,$821.8,$412.6,$409.3,http://www.boxofficemojo.com/movies/?id=wonder...


In [313]:
boxoffice=boxoffice[-boxoffice["Overseas"].isin(['n/a'])]
boxoffice=boxoffice[-boxoffice["Domestic"].isin(['n/a'])]

In [314]:
boxoffice=boxoffice.reset_index(drop=True) 
boxoffice.head()

,Rank,Title,Studio,Worldwide,Domestic,Overseas,Link
0,1,Star Wars: The Last Jedi,BV,"$1,331.1",$619.1,$712.0,http://www.boxofficemojo.com/movies/?id=starwa...
1,2,Beauty and the Beast (2017),BV,"$1,263.5",$504.0,$759.5,http://www.boxofficemojo.com/movies/?id=beauty...
2,3,The Fate of the Furious,Uni.,"$1,236.0",$226.0,"$1,010.0",http://www.boxofficemojo.com/movies/?id=furiou...
3,4,Despicable Me 3,Uni.,"$1,034.8",$264.6,$770.2,http://www.boxofficemojo.com/movies/?id=despic...
4,5,Jumanji: Welcome to the Jungle,Sony,$930.0,$393.1,$536.9,http://www.boxofficemojo.com/movies/?id=jumanj...


In [315]:
row=boxoffice.shape[0]
column=boxoffice.shape[1]

In [316]:
import re
for i in range(row):
    x=boxoffice["Worldwide"][i]
    split1=re.split(r'[$]',x)
    boxoffice["Worldwide"][i]=split1[1]

In [317]:
import re
for i in range(row):
    x=boxoffice["Domestic"][i]
    split1=re.split(r'[$]',x)
    boxoffice["Domestic"][i]=split1[1]


In [318]:
import re
for i in range(row):
    x=boxoffice["Overseas"][i]
    split1=re.split(r'[$]',x)
    boxoffice["Overseas"][i]=split1[1]

In [319]:
boxoffice.head()

,Rank,Title,Studio,Worldwide,Domestic,Overseas,Link
0,1,Star Wars: The Last Jedi,BV,"1,331.1",619.1,712.0,http://www.boxofficemojo.com/movies/?id=starwa...
1,2,Beauty and the Beast (2017),BV,"1,263.5",504.0,759.5,http://www.boxofficemojo.com/movies/?id=beauty...
2,3,The Fate of the Furious,Uni.,"1,236.0",226.0,"1,010.0",http://www.boxofficemojo.com/movies/?id=furiou...
3,4,Despicable Me 3,Uni.,"1,034.8",264.6,770.2,http://www.boxofficemojo.com/movies/?id=despic...
4,5,Jumanji: Welcome to the Jungle,Sony,930.0,393.1,536.9,http://www.boxofficemojo.com/movies/?id=jumanj...


In [320]:
import re
for i in range(row):
    x=boxoffice["Worldwide"][i]
    if x.endswith("k"):
        split1=re.split(r'[k]',x)
        boxoffice["Worldwide"][i]=float(float(split1[0])*0.01)

In [321]:
import re
for i in range(row):
    x=boxoffice["Domestic"][i]
    if x.endswith("k"):
        split1=re.split(r'[k]',x)
        boxoffice["Domestic"][i]=float(float(split1[0])*0.01)

In [322]:
import re
for i in range(row):
    x=boxoffice["Overseas"][i]
    if x.endswith("k"):
        split1=re.split(r'[k]',x)
        boxoffice["Overseas"][i]=float(float(split1[0])*0.01)


In [323]:
boxoffice.head()

,Rank,Title,Studio,Worldwide,Domestic,Overseas,Link
0,1,Star Wars: The Last Jedi,BV,"1,331.1",619.1,712.0,http://www.boxofficemojo.com/movies/?id=starwa...
1,2,Beauty and the Beast (2017),BV,"1,263.5",504.0,759.5,http://www.boxofficemojo.com/movies/?id=beauty...
2,3,The Fate of the Furious,Uni.,"1,236.0",226.0,"1,010.0",http://www.boxofficemojo.com/movies/?id=furiou...
3,4,Despicable Me 3,Uni.,"1,034.8",264.6,770.2,http://www.boxofficemojo.com/movies/?id=despic...
4,5,Jumanji: Welcome to the Jungle,Sony,930.0,393.1,536.9,http://www.boxofficemojo.com/movies/?id=jumanj...


In [324]:
#Make sure all columns of numbers are numeric (not strings).
boxoffice.Worldwide =pd.to_numeric(boxoffice.Worldwide, errors='coerce').fillna(0).astype(float)
boxoffice.Domestic =pd.to_numeric(boxoffice.Domestic, errors='coerce').fillna(0).astype(float)
boxoffice.Overseas =pd.to_numeric(boxoffice.Overseas,errors='coerce').fillna(0).astype(float)

In [325]:
boxoffice.dtypes

Rank          object
Title         object
Studio        object
Worldwide    float64
Domestic     float64
Overseas     float64
Link          object
dtype: object

In [326]:
boxoffice.head()

,Rank,Title,Studio,Worldwide,Domestic,Overseas,Link
0,1,Star Wars: The Last Jedi,BV,0.0,619.1,712.0,http://www.boxofficemojo.com/movies/?id=starwa...
1,2,Beauty and the Beast (2017),BV,0.0,504.0,759.5,http://www.boxofficemojo.com/movies/?id=beauty...
2,3,The Fate of the Furious,Uni.,0.0,226.0,0.0,http://www.boxofficemojo.com/movies/?id=furiou...
3,4,Despicable Me 3,Uni.,0.0,264.6,770.2,http://www.boxofficemojo.com/movies/?id=despic...
4,5,Jumanji: Welcome to the Jungle,Sony,930.0,393.1,536.9,http://www.boxofficemojo.com/movies/?id=jumanj...


In [327]:
boxoffice['Dome_minus_Over']=(boxoffice['Overseas']-boxoffice['Domestic'])/boxoffice['Domestic']
boxoffice.head()

,Rank,Title,Studio,Worldwide,Domestic,Overseas,Link,Dome_minus_Over
0,1,Star Wars: The Last Jedi,BV,0.0,619.1,712.0,http://www.boxofficemojo.com/movies/?id=starwa...,0.150057
1,2,Beauty and the Beast (2017),BV,0.0,504.0,759.5,http://www.boxofficemojo.com/movies/?id=beauty...,0.506944
2,3,The Fate of the Furious,Uni.,0.0,226.0,0.0,http://www.boxofficemojo.com/movies/?id=furiou...,-1.000000
3,4,Despicable Me 3,Uni.,0.0,264.6,770.2,http://www.boxofficemojo.com/movies/?id=despic...,1.910809
4,5,Jumanji: Welcome to the Jungle,Sony,930.0,393.1,536.9,http://www.boxofficemojo.com/movies/?id=jumanj...,0.365810


In [328]:
boxoffice=boxoffice[(boxoffice['Dome_minus_Over']>1)&(boxoffice['Dome_minus_Over']<60)]
boxoffice

,Rank,Title,Studio,Worldwide,Domestic,Overseas,Link,Dome_minus_Over
3,4,Despicable Me 3,Uni.,0.00,264.600,770.20,http://www.boxofficemojo.com/movies/?id=despic...,1.910809
10,11,Pirates of the Caribbean: Dead Men Tell No Tales,BV,794.90,172.600,622.30,http://www.boxofficemojo.com/movies/?id=potc5.htm,2.605446
11,12,Coco,BV,744.90,208.500,536.40,http://www.boxofficemojo.com/movies/?id=pixar1...,1.572662
15,16,Transformers: The Last Knight,Par.,605.40,130.200,475.30,http://www.boxofficemojo.com/movies/?id=transf...,2.650538
16,17,Kong: Skull Island,WB,566.70,168.100,398.60,http://www.boxofficemojo.com/movies/?id=legend...,1.371208
19,20,War for the Planet of the Apes,Fox,490.70,146.900,343.80,http://www.boxofficemojo.com/movies/?id=planet...,1.340368
20,21,Kingsman: The Golden Circle,Fox,410.80,100.200,310.60,http://www.boxofficemojo.com/movies/?id=kingsm...,2.099800
21,22,The Mummy (2017),Uni.,409.20,80.200,329.00,http://www.boxofficemojo.com/movies/?id=mummy2...,3.102244
23,24,Fifty Shades Darker,Uni.,381.10,114.600,266.50,http://www.boxofficemojo.com/movies/?id=fiftys...,1.325480
26,27,Murder on the Orient Express (2017),Fox,350.30,102.800,247.50,http://www.boxofficemojo.com/movies/?id=murder...,1.407588


Korea:CJ

In [329]:
selected=set(boxoffice['Studio'])
selected

{'Amazon',
 'BG',
 'BV',
 'CJ',
 'CL',
 'Distrib.',
 'Elev.',
 'FIP',
 'FR',
 'FUN',
 'Fox',
 'FoxS',
 'GrtIndia',
 'IFC',
 'Par.',
 'SGem',
 'STX',
 'Sony',
 'TriS',
 'Uni.',
 'WB',
 'WB (NL)',
 'WGUSA',
 'Wein.',
 'Yash'}

In [330]:
data=boxoffice[(boxoffice['Studio']=='Amazon')|(boxoffice['Studio']=='BG')|
              (boxoffice['Studio']=='BV')|(boxoffice['Studio']=='Fox')|
              (boxoffice['Studio']=='SGem')|(boxoffice['Studio']=='STX')|
              (boxoffice['Studio']=='Sony')|(boxoffice['Studio']=='Par.')|
              (boxoffice['Studio']=='Uni.')|(boxoffice['Studio']=='WB')|
              (boxoffice['Studio']=='WB (NL)')|(boxoffice['Studio']=='Wein.')]
data=data.reset_index(drop=True) 
data

,Rank,Title,Studio,Worldwide,Domestic,Overseas,Link,Dome_minus_Over
0,4,Despicable Me 3,Uni.,0.00,264.60,770.20,http://www.boxofficemojo.com/movies/?id=despic...,1.910809
1,11,Pirates of the Caribbean: Dead Men Tell No Tales,BV,794.90,172.60,622.30,http://www.boxofficemojo.com/movies/?id=potc5.htm,2.605446
2,12,Coco,BV,744.90,208.50,536.40,http://www.boxofficemojo.com/movies/?id=pixar1...,1.572662
3,16,Transformers: The Last Knight,Par.,605.40,130.20,475.30,http://www.boxofficemojo.com/movies/?id=transf...,2.650538
4,17,Kong: Skull Island,WB,566.70,168.10,398.60,http://www.boxofficemojo.com/movies/?id=legend...,1.371208
5,20,War for the Planet of the Apes,Fox,490.70,146.90,343.80,http://www.boxofficemojo.com/movies/?id=planet...,1.340368
6,21,Kingsman: The Golden Circle,Fox,410.80,100.20,310.60,http://www.boxofficemojo.com/movies/?id=kingsm...,2.099800
7,22,The Mummy (2017),Uni.,409.20,80.20,329.00,http://www.boxofficemojo.com/movies/?id=mummy2...,3.102244
8,24,Fifty Shades Darker,Uni.,381.10,114.60,266.50,http://www.boxofficemojo.com/movies/?id=fiftys...,1.325480
9,27,Murder on the Orient Express (2017),Fox,350.30,102.80,247.50,http://www.boxofficemojo.com/movies/?id=murder...,1.407588


In [331]:
title=list(boxoffice['Title'])
title

['Despicable Me 3',
 'Pirates of the Caribbean: Dead Men Tell No Tales',
 'Coco',
 'Transformers: The Last Knight',
 'Kong: Skull Island',
 'War for the Planet of the Apes',
 'Kingsman: The Golden Circle',
 'The Mummy (2017)',
 'Fifty Shades Darker',
 'Murder on the Orient Express (2017)',
 'xXx: The Return of Xander Cage',
 'The Great Wall',
 'Resident Evil: The Final Chapter',
 'Annabelle: Creation',
 'Ferdinand',
 'Journey to the West: The Demons Strike Back',
 'Alien: Covenant',
 'Valerian and the City of a Thousand Planets',
 'Geostorm',
 'Smurfs: The Lost Village',
 "A Dog's Purpose",
 'Baywatch',
 'Ghost in the Shell (2017)',
 'Duckweed',
 'King Arthur: Legend of the Sword',
 'The Foreigner',
 'Baahubali 2: The Conclusion',
 'Buddies in India',
 'Leap!',
 'Railroad Tigers',
 'Life (2017)',
 'Tiger Zinda Hai',
 'A Taxi Driver',
 'One Piece Film: Gold',
 'Confidential Assignment',
 'The Battleship Island',
 'The Snowman',
 'T2: Trainspotting',
 'Judwaa 2',
 'A Cure for Wellness',


In [233]:
import requests 
import re
import pandas as pd

#使用requests获取网页信息
def getHTMLText(url):
    try:
        r=requests.get(url,timeout=30)
        r.raise_for_status()
        r.encoding=r.apparent_encoding 
        return r.text
    except:
        return ''

#使用正则表达式提取目标信息
def parsePage(ilt,html):
    try:
        plt=re.findall('<td align="center">.*?</td>',html)
        l=len(plt)
        l=l//7
        for i in range(l):
            j=i*7
            num=plt[j].split('">')[1].split('<')[0]
            name=plt[j+1].split('">')[1].split('<')[0]
            loc=plt[j+2].split('&nbsp;')[1].split('<')[0]
            signNum=plt[j+3].split('">')[1].split('<')[0]
            bookNum=plt[j+4].split('">')[1].split('<')[0]
            area=plt[j+5].split('">')[1].split('<')[0].replace('&nbsp;', '') 
            avePrice=plt[j+6].split('">')[1].split('<')[0]
            ilt.append([num,name,loc,signNum,bookNum,area,avePrice]) 
    except:
        return ''
    
#保存为csv文件      
def saveGoodslist(ilt):
    pd.DataFrame(ilt).to_csv('housing_transcation.csv',index=False,header=False)
        
def main():
    infolist=[]
    infolist.append([ '序号','楼盘名称','城区','签约套数','预定套数','签约面积','签约均价'])
    url='http://hz.house.ifeng.com/detail/2014_10_28/50087618_1.shtml'
    html=getHTMLText(url)
    parsePage(infolist,html)      
    saveGoodslist(infolist)
    
main()

In [11]:
td_list = bsObj.find_all('td')  # 这里返回的是h2标签的list列表
td_list

[<td class="previous"><a href="/box-office-records/worldwide/all-movies/cumulative/released-in-2016">Top 2016 Worldwide</a></td>,
 <td class="index"></td>,
 <td class="next"><a href="/box-office-records/worldwide/all-movies/cumulative/released-in-2018">Top 2018 Worldwide</a></td>,
 <td class="data">1</td>,
 <td><b><a href="/movie/Star-Wars-Ep-VIII-The-Last-Jedi#tab=summary">Star Wars Ep. VIII: The Last Jedi</a></b></td>,
 <td align="right">$1,323,242,625</td>,
 <td align="right">$618,229,777</td>,
 <td align="right">$705,012,848</td>,
 <td align="right">46.72%</td>,
 <td class="data">2</td>,
 <td><b><a href="/movie/Beauty-and-the-Beast-(2017)#tab=summary">Beauty and the Beast</a></b></td>,
 <td align="right">$1,263,109,573</td>,
 <td align="right">$504,014,165</td>,
 <td align="right">$759,095,408</td>,
 <td align="right">39.90%</td>,
 <td class="data">3</td>,
 <td><b><a href="/movie/Fate-of-the-Furious-The#tab=summary">The Fate of the Furious</a></b></td>,
 <td align="right">$1,237,46

In [6]:
import urllib.request
from bs4 import BeautifulSoup
import time

url = 'https://www.the-numbers.com/box-office-records/worldwide/all-movies/cumulative/released-in-2017'  
html = urllib.request.urlopen(url)    
bsObj = BeautifulSoup(html,'lxml')
bsObj


<!DOCTYPE html>
<html>
<head>
<meta content='(PICS-1.1 "https://www.icra.org/ratingsv02.html" l gen true for "https://www.the-numbers.com/" r (cb 1 lz 1 nz 1 oz 1 vz 1) "https://www.rsac.org/ratingsv01.html" l gen true for "https://www.the-numbers.com/" r (n 0 s 0 v 0 l 0))' http-equiv="PICS-Label"/>
<!--<meta http-equiv="Content-Type" content="text/html; charset=ISO-8859-1" >-->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="telephone=no" name="format-detection"/> <!-- for apple mobile -->
<meta content="521546213" property="fb:admins"/>
<meta content="initial-scale=1" name="viewport"/>
<meta content="This chart contains the total worldwide box office for the top 100 movies released in 2017." name="description"/>
<meta content="NOODP" name="robots"/>
<meta content="Worldwide, Movie, Box Office, Records, All Movies, Released in 2017, Cumulative" name="keywords"/>
<meta content="Top 2017 Movies at the Worldwide Box Office" property="og:title"/>
<meta

In [2]:
from bs4 import BeautifulSoup
from urllib2 import urlopen
import csv
import re


current_url=("http://www.boxofficemojo.com/movies/alphabetical.htm?letter=NUM&p=.html")# starting point for search, can be any letter
movie_links=[]#initialize as an empty list


soup = BeautifulSoup(urlopen(current_url).read())	#generate list of links for the letter indices
letters = soup.findAll('a', href= re.compile('letter='))
letter_index=[] #intialize as an empty list
for t in letters:
	letter_index.append("http://www.boxofficemojo.com" + t['href'])

for i in range(0,27): #loop through all letter indices for movies
	current_url=letter_index[i]
	soup = BeautifulSoup(urlopen(current_url).read())
	navbar=soup.find('div', 'alpha-nav-holder')
	pages = navbar.findAll('a', href= re.compile('alphabetical'))
	page_list=[] # page_list is reset for each letter index

	for t in pages:
		page_list.append("http://www.boxofficemojo.com" + t['href'])


	movietable=soup.find('div',{'id':'main'})
	movies = movietable.findAll('a', href= re.compile('id='))
	for t in movies:
		movie_links.append("http://www.boxofficemojo.com" + t['href'])

	if pages != None: #this only runs if there is a 2nd page for this letter				
		i=0	#page list starts at 2 (consequence of page layout)
		while i<len(page_list): #loop over multiple pages for each letter index
			current_url=page_list[i] 
			soup = BeautifulSoup(urlopen(current_url).read())
			movietable=soup.find('div',{'id':'main'})
			movies = movietable.findAll('a', href= re.compile('id='))
			for t in movies:
				movie_links.append("http://www.boxofficemojo.com" + t['href'])
			i+=1


with open("/home/christopher/data_science/movie_data.csv", "w") as f:
	fieldnames = ("title","director1","director2","domestic", "distributor","release","genre","runtime","rating","budget","worldwide","actor1","actor2","actor3","actor4","actor5","actor6","producer1","producer2","producer3","producer4","producer5","producer6","writer1","writer2","composer1","composer2")#can add foreign release dates/boxoffice, review data, award data, trope data from other sites eventually
	output = csv.writer(f, delimiter=",")
	output.writerow(fieldnames)

	for url in movie_links:
		if "elizabeth" in url and "elizabethtown" not in url:#fixes an annoying encoding error in an inelagent way
			url='http://www.boxofficemojo.com/movies/?id=elizabeth%A0.htm'
		if "simpleplan" in url:
			url='http://www.boxofficemojo.com/movies/?id=simpleplan%A0.htm'
		print url
		#time.sleep(0.1) #pause for courtesy? not sure if neccesary,i'm new at this
		current_url = (url + "&adjust_yr=2015&p=.htm")  #do all movies in 2015 dollars (done automatically by site with correct URL)
	
		soup = BeautifulSoup(urlopen(current_url).read())

		directors=soup.findAll('a', href= re.compile('Director&id'))
		director_list=[]
		for t in directors:
			director_list.append(t.encode_contents())
		for i in range(0,2):
			if i>=len(director_list):
				director_list.append('N/A')#fill rest of list
		director1=director_list[0]
		director2=director_list[1]

		writers=soup.findAll('a', href= re.compile('Writer&id'))
		writer_list=[]
		for t in writers:
			writer_list.append(t.encode_contents())
		for i in range(0,2):
			if i>=len(writer_list):
				writer_list.append('N/A')
		writer1=writer_list[0]
		writer2=writer_list[1]

		composers=soup.findAll('a', href= re.compile('Composer&id'))
		composer_list=[]
		for t in composers:
			composer_list.append(t.encode_contents())
		for i in range(0,2):
			if i>=len(composer_list):
				composer_list.append('N/A')
		composer1=composer_list[0]
		composer2=composer_list[1]

		actors=soup.findAll('a', href= re.compile('Actor&id'))
		actor_list=[]
		for t in actors:
			actor_list.append(t.encode_contents())
		for i in range(0,6):
			if i>=len(actor_list):
				actor_list.append('N/A')
		actor1=actor_list[0]
		actor2=actor_list[1]
		actor3=actor_list[2]
		actor4=actor_list[3]
		actor5=actor_list[4]
		actor6=actor_list[5]
	
		producers=soup.findAll('a', href= re.compile('Producer&id'))
		producer_list=[]
		for t in producers:
			producer_list.append(t.encode_contents())
		for i in range(0,6):
			if i>=len(producer_list):
				producer_list.append('N/A')
		producer1=producer_list[0]
		producer2=producer_list[1]
		producer3=producer_list[2]
		producer4=producer_list[3]
		producer5=producer_list[4]
		producer6=producer_list[5]
	

		all_bs=soup.findAll('b')
		b_list=[] #lots of the information we want is in bold, and appears in the same order on each page
		for t in all_bs:
			if 'Domestic Lifetime' not in t.encode_contents():#want to ignore the lifetime box office
				b_list.append(t.encode_contents())
		if len(b_list)>=10:#avoids bad entries with no box office data
			if '$'in b_list[2] or 'n/a' in b_list[9]:#avoid movies w/o box office data, or unadjustable box office data, if not caught above
				if 'n/a' in b_list[9]:#has a foreign release only, order is shifted
					title=b_list[1]
					domestic='N/A'
					if 'N/A' not in b_list[2]:
						distributor=b_list[2].split('>')[1].split('<')[0]
					else:
						distributor=b_list[2]
					if len(b_list[3].split('>'))>3:#sometimes the release date is not in a hyperlink
						release=b_list[3].split('>')[2].split('<')[0]
					else:
						release=b_list[3].split('>')[1].split('<')[0]
					genre=b_list[4]
					runtime=b_list[5]
					rating=b_list[6]
					budget=b_list[7]
					worldwide=b_list[12]
				else:	#has a domestic release
					title=b_list[1]
					domestic=b_list[2]
					if 'n/a' not in b_list[3]:
						distributor=b_list[3].split('>')[1].split('<')[0]
					else:
						distributor=b_list[3]
					if len(b_list[4].split('>'))>3:#sometimes the release date is not in a hyperlink
						release=b_list[4].split('>')[2].split('<')[0]
					else:
						release=b_list[4].split('>')[1].split('<')[0]
					genre=b_list[5]
					runtime=b_list[6]
					rating=b_list[7]
					budget=b_list[8]
					if len(b_list)==11 or '%' not in b_list[11]:#this means it only has a domestic release
						worldwide='N/A'
					else:
						worldwide=b_list[13]
				#print release
				output.writerow([title,director1,director2,domestic,distributor,release,genre,runtime,rating,budget,worldwide,actor1,actor2,actor3,actor4,actor5,actor6,producer1,producer2,producer3,producer4,producer5,producer6,writer1,writer2,composer1,composer2])#since this is in the big "if" it wont write to file if it is formated incorrectly
		
		#else:
			#print 'bad format'
			#print url	#i want to see which ones i threw away, to check if my criteria is good
			#print 'bad format'

print "Done writing file"

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(t url)? (<ipython-input-2-27df1a96613c>, line 55)

In [5]:
import boxofficemojoAPI as bom

box_office_mojo = bom.BoxOfficeMojo()
box_office_mojo.crawl_for_urls()


movie = box_office_mojo.get_movie_summary("titanic")
movie.clean_data()
print(movie.to_json())

weekly = box_office_mojo.get_weekly_summary("titanic")
weekly.clean_data()
print(weekly.to_json())

ModuleNotFoundError: No module named 'boxofficemojoAPI'